# Descripcion

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.
Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del curso de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.
Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
El proyecto estara enfocado en encontrar el modelo de prediccion que mas se ajuste a la tarea que `Megaline` nos ha asignado, veremos como los diferentes proyectos se desenvuelven usando este dataframe, mientras los ajustaremos y evaluaremos para escoger el indicado para la tarea requerida, al final haremos una prueba de cordura para estar `100%` seguro de haber escogido el modelo adecuado para la tarea y poder llegar a las conclusiones necesarias para completar el trabajo adecuadamente.
</div>

## Inicialización

In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

## Cargar los Datos

In [2]:
# Carga el archivo de datos en un DataFrame

try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

## Explorar datos iniciales

La información dada es la siguiente:
- `сalls` — número de llamadas,
- `minutes` — duración total de la llamada en minutos,
- `messages` — número de mensajes de texto,
- `mb_used` — Tráfico de Internet utilizado en MB,
- `is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0).

<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Aca hemos cargado las librerias necesarias para el proyecto, cargamos el dataframe que usaremos y definimos el tipo de informacion que contiene el dataframe a ser usado.
</div>

In [3]:
# imprime la información general/resumida sobre el DataFrame

df.shape

(3214, 5)

In [4]:
# imprimir una muestra de datos

df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [5]:
# Verificar si hay valores nulos en cada columna

null_values = df.isnull().sum()

print("Valores nulos por columna:")
null_values

Valores nulos por columna:


calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Como podemos ver no hay valores nulos existente en el dataframe, por lo cual procederemos a por conveniencia ajustar el tipo de datos contenidos en este, por ejemplo el hecho de que las columnas `calls` y `messages` no tengan valores enteros debemos corregirlo para mayor facilidad de uso, al igual que en el caso de las columnas `minutes` y `mb_used`, estas las redondearemos hacia arriba para trabajar unicamente con numeros enteros.
</div>

In [6]:
# Ajustar los tipos de datos y redondear las columnas

df['calls'] = df['calls'].astype(int)
df['messages'] = df['messages'].astype(int)
df['minutes'] = np.ceil(df['minutes']).astype(int)  # Redondear hacia arriba
df['mb_used'] = np.ceil(df['mb_used']).astype(int)  # Redondear hacia arriba

# Mostrar las primeras filas del DataFrame para verificar los cambios
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40,312,83,19916,0
1,85,517,56,22697,0
2,77,468,86,21061,0
3,106,746,81,8438,1
4,66,419,1,14503,0
5,58,345,21,15824,0
6,57,432,20,3739,1
7,15,133,6,21912,0
8,7,44,3,2539,1
9,90,666,38,17359,0


<div class="alert alert-block alert-info">

<b>Respuesta del estudiante, al comentario del revisor</b> <a class="tocSkip"></a>
Aca he corregido la forma de aplicar la funcion `np.ceil` usandola directamente en las columnas, sin usar el metodo `apply` como se me requirio.
</div>

## Segmentacion Dataframe antes de trabajar con los modelos predictivos

In [7]:
# Dividir el DataFrame en conjunto de entrenamiento (60%), validación (20%) y pruebas (20%)

train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)
validation_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# Verificar las formas (tamaños) de los conjuntos resultantes
print("Entrenamiento:", train_df.shape)
print("Validación:", validation_df.shape)
print("Pruebas:", test_df.shape)

Entrenamiento: (1928, 5)
Validación: (643, 5)
Pruebas: (643, 5)


<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
La segmentacion que fue realizada en los datos cumple con lo aprendido previamente para estandarizar los resultados, dividir el `dataframe` en partes `3:1:1`, en este caso `60%: Entrenamiento`, `20%: Validacion` y `20%: Pruebas`, ahora vamos a investigar la calidad de los diferentes modelos aprendidos para saber con cual podriamos seleccionar que nos ayude a cumplir el cometido de la mejor manera posible.
</div>

## Elegir modelos y configurar hiperparámetros

In [8]:
# Crear un modelo de árbol de decisión para clasificación
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo con train_df
decision_tree_classifier.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# El modelo está entrenado y listo para su uso

DecisionTreeClassifier(random_state=42)

In [9]:
# Crear un modelo de regresión logística
logistic_regression = LogisticRegression(random_state=42)

# Entrenar el modelo con train_df
logistic_regression.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# El modelo está entrenado y listo para su uso

LogisticRegression(random_state=42)

In [10]:
# Crear un modelo de bosque aleatorio para clasificación
random_forest_classifier = RandomForestClassifier(random_state=42)

# Entrenar el modelo con train_df
random_forest_classifier.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# El modelo está entrenado y listo para su uso

RandomForestClassifier(random_state=42)

<div class="alert alert-block alert-info">

<b>Respuesta del estudiante CORREGIDO</b> <a class="tocSkip"></a>
Aca he hecho la correccion sugerida para utilizar unicamente los modelos que funcionarian correctamente en el caso que se esta presentando en este proyecto especificamente para `Megaline`.
</div>

<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Como se puede observar dimos uso a todos los modelos que funcionan con el caso especifico que tenemos en `Megaline`, a pesar de que cada uno tiene diferencias con los demas, para el proposito que haremos uso esta vez, es mas que valido, ahora que vemos que los modelos estan entrenados, vamos a validarlos y ajustarlos antes de evaluarlos.
</div>

## Entrenar y ajustar los modelos

In [11]:
# Evaluar el modelo Decision Tree Classifier en el conjunto de validación
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Definir los hiperparámetros a probar
param_grid_decision_tree = {
    'max_depth': [3, 5, 7, 10]  # Aquí puedes agregar más valores para probar
}

# Realizar la búsqueda en cuadrícula
grid_search_decision_tree = GridSearchCV(decision_tree_classifier, param_grid_decision_tree, cv=5)
grid_search_decision_tree.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para Decision Tree Classifier:", grid_search_decision_tree.best_params_)

# Evaluar el modelo en el conjunto de validación con los mejores hiperparámetros encontrados
decision_tree_classifier_score = grid_search_decision_tree.best_estimator_.score(validation_df.drop('is_ultra', axis=1), validation_df['is_ultra'])
print("Desempeño del modelo Decision Tree Classifier en el conjunto de validación:", decision_tree_classifier_score)

Mejores hiperparámetros para Decision Tree Classifier: {'max_depth': 3}
Desempeño del modelo Decision Tree Classifier en el conjunto de validación: 0.7931570762052877


In [12]:
# Evaluar el modelo Logistic Regression en el conjunto de validación
logistic_regression = LogisticRegression(random_state=42)

# Definir los hiperparámetros a probar
param_grid_logistic_regression = {
    'C': [0.1, 1, 10, 100]  # Aquí puedes agregar más valores para probar
}

# Realizar la búsqueda en cuadrícula
grid_search_logistic_regression = GridSearchCV(logistic_regression, param_grid_logistic_regression, cv=5)
grid_search_logistic_regression.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para Logistic Regression:", grid_search_logistic_regression.best_params_)

# Evaluar el modelo en el conjunto de validación con los mejores hiperparámetros encontrados
logistic_regression_score = grid_search_logistic_regression.best_estimator_.score(validation_df.drop('is_ultra', axis=1), validation_df['is_ultra'])
print("Desempeño del modelo Logistic Regression en el conjunto de validación:", logistic_regression_score)

Mejores hiperparámetros para Logistic Regression: {'C': 0.1}
Desempeño del modelo Logistic Regression en el conjunto de validación: 0.7402799377916018


In [13]:
# Evaluar el modelo Random Forest Classifier en el conjunto de validación
random_forest_classifier = RandomForestClassifier(random_state=42)

# Definir los hiperparámetros a probar
param_grid_random_forest = {
    'n_estimators': [100, 200, 300],  # Aquí puedes agregar más valores para probar
    'max_depth': [3, 5, 7, None]  # Aquí puedes agregar más valores para probar
}

# Realizar la búsqueda en cuadrícula
grid_search_random_forest = GridSearchCV(random_forest_classifier, param_grid_random_forest, cv=5)
grid_search_random_forest.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros para Random Forest Classifier:", grid_search_random_forest.best_params_)

# Evaluar el modelo en el conjunto de validación con los mejores hiperparámetros encontrados
random_forest_classifier_score = grid_search_random_forest.best_estimator_.score(validation_df.drop('is_ultra', axis=1), validation_df['is_ultra'])
print("Desempeño del modelo Random Forest Classifier en el conjunto de validación:", random_forest_classifier_score)

Mejores hiperparámetros para Random Forest Classifier: {'max_depth': None, 'n_estimators': 200}
Desempeño del modelo Random Forest Classifier en el conjunto de validación: 0.8087091757387247


<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Los modelos fueron entrenados utilizando el conjunto de entrenamiento y validacion para mayor precision y eficacia al momento de hacer las pruebas de exactitud, como podemos observar de los modelos que estamos probando vemos que se va perfilando el modelo `Random_Forest_Classifier` como el mas adecuado para la tarea que queremos realizar, ahora vamos a evaluar la precision de los modelos.
</div>

## Evaluar la calidad de los modelos

In [15]:
# Evaluar el modelo Decision Tree Classifier en el conjunto de prueba
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Ajustar el modelo con los datos de entrenamiento
decision_tree_classifier.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# Evaluar el modelo en el conjunto de prueba
decision_tree_classifier_score_test = decision_tree_classifier.score(test_df.drop('is_ultra', axis=1), test_df['is_ultra'])

print("Desempeño del modelo Decision Tree Classifier en el conjunto de prueba:", decision_tree_classifier_score_test)

Desempeño del modelo Decision Tree Classifier en el conjunto de prueba: 0.7356143079315708


In [16]:
# Evaluar el modelo Logistic Regression en el conjunto de prueba
logistic_regression = LogisticRegression(random_state=42)

# Ajustar el modelo con los datos de entrenamiento
logistic_regression.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# Evaluar el modelo en el conjunto de prueba
logistic_regression_score_test = logistic_regression.score(test_df.drop('is_ultra', axis=1), test_df['is_ultra'])

print("Desempeño del modelo Logistic Regression en el conjunto de prueba:", logistic_regression_score_test)

Desempeño del modelo Logistic Regression en el conjunto de prueba: 0.7682737169517885


In [17]:
# Evaluar el modelo Random Forest Classifier en el conjunto de prueba
random_forest_classifier = RandomForestClassifier(random_state=42)

# Ajustar el modelo con los datos de entrenamiento
random_forest_classifier.fit(train_df.drop('is_ultra', axis=1), train_df['is_ultra'])

# Evaluar el modelo en el conjunto de prueba
random_forest_classifier_score_test = random_forest_classifier.score(test_df.drop('is_ultra', axis=1), test_df['is_ultra'])

print("Desempeño del modelo Random Forest Classifier en el conjunto de prueba:", random_forest_classifier_score_test)

Desempeño del modelo Random Forest Classifier en el conjunto de prueba: 0.8133748055987559


<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Las evaluaciones de calidad no nos defraudaron, la misma tendencia que vimos mientras entrenabamos y ajustabamos los modelos la podemos apreciar en la evaluacion, asi que al final vamos a utilizar `Random_Forest_Classifier` como nuestro modelo ganador, procederemos a validar la exactitud del modelo antes de proceder a la prueba de cordura.
</div>

In [18]:
umbral_exactitud = 0.75

if random_forest_classifier_score_test > umbral_exactitud:
    print(f"¡El modelo Random Forest Classifier ha alcanzado un rendimiento de {random_forest_classifier_score_test} en el conjunto de prueba, superando el umbral requerido de {umbral_exactitud}!")
else:
    print(f"El modelo Random Forest Classifier no alcanza el umbral requerido de {umbral_exactitud} en el conjunto de prueba, con un rendimiento de {random_forest_classifier_score_test}. Se recomienda realizar más ajustes o considerar otros enfoques.")

¡El modelo Random Forest Classifier ha alcanzado un rendimiento de 0.8133748055987559 en el conjunto de prueba, superando el umbral requerido de 0.75!


<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Luego de validar la certeza de que estamos ante un modelo que cumple de sobra con los requerimientos que nos impuso la empresa, vamos a hacer una prueba de cordura.
</div>

<div class="alert alert-block alert-info">

<b>Respuesta del estudiante, al comentario del revisor</b> <a class="tocSkip"></a>
En esta parte, he eliminado los modelos innecesarios como se me sugirio, luego en la optimizacion de hiperparametros utilice las variables de `entrenamiento` y `validacion` como me fue sugerido, para obtener unos resultados mas apropiados.
</div>

## Prueba de cordura con datos sinteticos

In [19]:
# Supongamos que tienes un cliente extremadamente inusual con valores altos
datos_cliente_inusual = {
    'calls': 300,
    'minutes': 2000,
    'messages': 100,
    'mb_used': 50000,
    'is_ultra': None  # Esta será la predicción que queremos obtener del modelo
}

# Realiza la predicción con el modelo
nuevos_datos = [datos_cliente_inusual['calls'], datos_cliente_inusual['minutes'],
                datos_cliente_inusual['messages'], datos_cliente_inusual['mb_used']]
prediccion = random_forest_classifier.predict([nuevos_datos])

# Imprime la predicción del modelo
print("Predicción del modelo para datos inusuales:", prediccion)

Predicción del modelo para datos inusuales: [1]


<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Con esta prueba queda evidenciada la eficacia del modelo ya que ciertamente los datos sinteticos provistos corresponden al perfil de un cliente perteneciente al plan Ultra, designado con el numero `1`, digase que podemos proceder a sacar conclusiones de la investigacion.
</div>

## Conclusiones

<div class="alert alert-block alert-info">

<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
Al tener unos datos concluyentes y un resultado bastante satisfactorio podemos asumir que el tratar los datos con el modelo seleccionado sera de gran beneficio para `Megaline`, ya que con esto se podra automatizar la clasificacion de clientes y se podra ofrecer de manera mas precisa el plan que mas se ajuste al uso que muestra el cliente en cuestion, tambien, podemos concluir que para este tipo de tareas es mas adecuado usar el modelo `Random_Forest_Classifier` ya que se adapta mejor que los demas a la tarea dandonos una eficacia de `~81%` en sus predicciones.
</div>